In [4]:
pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
from sklearn.preprocessing import OneHotEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
import tensorflow as tf
from tensorflow import keras


from keras.layers import Dense, LSTM, Embedding
from keras.models import Sequential
import os
import matplotlib.pyplot as plt

import gensim
import re
from numpy import array
from numpy import asarray
from numpy import zeros

from keras.models import Sequential
from keras.layers import Dense

from keras.layers import Embedding
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,GlobalMaxPooling1D

from keras import backend as K
from kerastuner import HyperModel
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

from tensorflow.keras.callbacks import EarlyStopping
from keras import regularizers
from tensorflow.keras.layers import BatchNormalization



C:\Users\hager\AppData\Local\Temp/ipykernel_25924/3664098320.py:42: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
word2vemodel = gensim.models.Word2Vec.load('/content/drive/MyDrive/ArabicSentimentHager/full_grams_cbow_300_twitter.mdl')

## Read training dataset

In [ ]:
#read train dataset
train= pd.read_csv('/content/drive/MyDrive/ArabicSentimentHager/paper2/Deep/sg/dataset1/Training_Main-AHS.csv',encoding='utf-8')
train=train.dropna()

y_train1 = train.label                           
X_train= train.drop('label',axis = 1 )

train.label.value_counts()

In [ ]:
#read testing dataset
test= pd.read_csv('/content/drive/MyDrive/ArabicSentimentHager/paper2/Deep/sg/dataset1/Testing_Main-AHS.csv',encoding='utf-8')
test=test.dropna()

y_test1= test.label                           
X_test= test.drop('label',axis = 1 )
test.label.value_counts()

## convert label to categorical

In [ ]:
#convert label to categorical
encoder_train_y = LabelEncoder()
y_train= encoder_train_y.fit_transform(y_train1)
y_train_categorical = np_utils.to_categorical(y_train)

encoder_test_y = LabelEncoder()
y_test = encoder_test_y.fit_transform(y_test1)
y_test_categorical=np_utils.to_categorical(y_test)

## Convert x_train and X_test into number

In [ ]:
MAX_LEN = 140 
MAX_NUM_WORDS = 200000
EMBEDDING_DIM = 300


tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(X_train['text'])
train_sequences = tokenizer.texts_to_sequences(X_train['text'])

word_index = tokenizer.word_index
padded_train = pad_sequences(sequences=train_sequences, maxlen=MAX_LEN)

test_sequences = tokenizer.texts_to_sequences(X_test['text'])
padded_test = pad_sequences(sequences=test_sequences, maxlen=MAX_LEN)

print('Total unique tokens generated: ',len(word_index))
print('Shape of padded train tensor: ', padded_train.shape)
print('Shape of padded test tensor: ', padded_test.shape)

## Bulid embedding_matrix

In [ ]:
num_words = min(MAX_NUM_WORDS, len(word_index))
word_embedding_matrix = np.zeros((num_words + 1, EMBEDDING_DIM))

for word, index in word_index.items():
    if index > MAX_NUM_WORDS:
        continue
    # embedding_vector = model[word]
    #word = clean_str(word)
    if word not in word2vemodel.wv:
        embedding_vector = None
    else:
        embedding_vector = word2vemodel.wv[word]
    if embedding_vector is not None:
        word_embedding_matrix[index] = embedding_vector

## Define some of function that are used as matixs 

In [ ]:

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Optimize and build mode 

In [ ]:

def build_model(hp):
    num_units_min  =  50
    num_units_max  =  1000
    num_units_step =  50

    dropout_min  =  .1
    dropout_max  =  0.9
    dropout_step =  0.1
    model = keras.Sequential()
    
    model.add(layers.Embedding(num_words+1, EMBEDDING_DIM, weights=[word_embedding_matrix], input_length=MAX_LEN))
    
    model.add(
        Conv1D(
            filters=hp.Choice(
                'num_filters1',[128,256] ),
            activation='relu',
            kernel_size=hp.Choice('kernel_size1', [2, 3,4,5])))
  
    model.add(MaxPooling1D(pool_size=hp.Choice('Pool_Size1', [2, 3,4,5])))

   

    model.add(
        Conv1D(
            filters=hp.Choice(
                'num_filters2',[128,256] ),
            activation='relu',
            kernel_size=hp.Choice('kernel_size2', [4,5])))
   

    model.add(MaxPooling1D(pool_size=hp.Choice('Pool_Size2', [4,5])))

  
    model.add(
        Conv1D(
            filters=hp.Choice(
                'num_filters3',[128,256,512] ),
            activation='relu',
            kernel_size=hp.Choice('kernel_size3', [2, 3,4,5])))    
    model.add(GlobalMaxPooling1D()  )
    
 

    

    model.add(Flatten())
    model.add(layers.Dense(units=hp.Int('unit1',  min_value=num_units_min,
                                                 max_value=num_units_max,
                                                 step=num_units_step),activation = 'relu'))
    #model.add(Dropout(rate=hp.Float('dropout4', min_value=dropout_min, max_value=dropout_max , default=dropout_step)))

    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer=keras.optimizers.Adam( hp.Float('learning_rate',min_value=1e-4,max_value=1e-2)),loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m] )
    return model



In [ ]:

tuner = RandomSearch(
    build_model,
    objective='val_acc',
    max_trials=10,
    project_name='/content/drive/MyDrive/ArabicSentimentHager/paper2/Deep/CBOW/dataset1/CNN')


In [ ]:
epoch=20
BATCH_SIZE=1000
early_stopping = EarlyStopping(monitor='val_acc', patience=30, verbose=1)
callback_list = [ early_stopping ]

# split training data into stratified train/dev sets

h=tuner.search(padded_train, y_train_categorical,
             epochs=epoch,
             batch_size=BATCH_SIZE, 
             callbacks=callback_list, validation_data=(padded_test,y_test_categorical) )
            
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.save("/content/drive/MyDrive/ArabicSentimentHager/paper2/Deep/CBOW/dataset1/model/CNN.h5")

In [ ]:
history=model.fit(padded_train,
                  y_train_categorical,
                  batch_size=BATCH_SIZE,
                  epochs=epoch, validation_data=(padded_test,y_test_categorical))


In [ ]:
print(history.history.keys())
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
AccuracyTest=[]
PrecisionTest=[]
RecallTest=[]
F1Test=[]
for i in range(0,1): 
    model.fit(padded_train, y_train_categorical, epochs=epoch, batch_size=BATCH_SIZE,  verbose=1, shuffle=True)
    y_p = model.predict(padded_test)
    y_pred= np.argmax(y_p, axis=1)

    Accurcy_Test= accuracy_score(y_test,y_pred)
    Precision_Test=precision_score(y_test, y_pred,average='weighted')

    Recall_Test=recall_score(y_test, y_pred, average='weighted')
    F1_Test=f1_score(y_test, y_pred, average='weighted') 

    AccuracyTest.append(round(100*Accurcy_Test, 2))
    PrecisionTest.append(round(100*Precision_Test, 2))
    RecallTest.append(round(100*Recall_Test, 2))
    F1Test.append(round(100*F1_Test, 2))

In [ ]:
ReultofTest=pd.DataFrame([])
ReultofTest=ReultofTest.append({'AccuracyTest' : round(np.mean(AccuracyTest),2),'PrecisionTest':round(np.mean(PrecisionTest),2),
             'RecallTest' : round(np.mean(RecallTest),2),'F1Test':round(np.mean(F1Test),2)}, ignore_index=True)

ReultofTest.reindex(['AccuracyTest','PrecisionTest','RecallTest','F1Test'], axis=1)